In [111]:
import scanpy as sc
import numpy as np
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score


In [44]:
latent = sc.read('./data/11_latent_moe.h5ad')
adata = sc.read('./data/09_tcr_annotation_A_B_with_gender_data.h5ad')

In [6]:
latent.uns['iroot'] = 36 #can comment this in and out

In [45]:
adata.uns['iroot'] = 36 #can comment this in and out

In [33]:
#sc.pp.neighbors(latent, n_neighbors=20, use_rep='X', method='gauss')
sc.pp.neighbors(latent, n_neighbors=5, method='gauss', knn=False)
sc.tl.diffmap(latent)
sc.tl.dpt(latent, n_branchings=1, n_dcs=10)

Index(['batch', 'Sample', 'donor', 'organ', 'sort', 'method', 'file',
       'is_TRA_p', 'is_TRB_p', 'is_TRA_np', 'is_TRB_np', 'Age', 'Source',
       'cell types', 'n_counts', 'log_counts', 'n_genes', 'mt_fraction',
       'multi_chain', 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus',
       'IR_VDJ_2_locus', 'IR_VJ_1_cdr3', 'IR_VJ_2_cdr3', 'IR_VDJ_1_cdr3',
       'IR_VDJ_2_cdr3', 'IR_VJ_1_cdr3_nt', 'IR_VJ_2_cdr3_nt',
       'IR_VDJ_1_cdr3_nt', 'IR_VDJ_2_cdr3_nt', 'IR_VJ_1_expr', 'IR_VJ_2_expr',
       'IR_VDJ_1_expr', 'IR_VDJ_2_expr', 'IR_VJ_1_expr_raw',
       'IR_VJ_2_expr_raw', 'IR_VDJ_1_expr_raw', 'IR_VDJ_2_expr_raw',
       'IR_VJ_1_v_gene', 'IR_VJ_2_v_gene', 'IR_VDJ_1_v_gene',
       'IR_VDJ_2_v_gene', 'IR_VJ_1_d_gene', 'IR_VJ_2_d_gene',
       'IR_VDJ_1_d_gene', 'IR_VDJ_2_d_gene', 'IR_VJ_1_j_gene',
       'IR_VJ_2_j_gene', 'IR_VDJ_1_j_gene', 'IR_VDJ_2_j_gene',
       'IR_VJ_1_c_gene', 'IR_VJ_2_c_gene', 'IR_VDJ_1_c_gene',
       'IR_VDJ_2_c_gene', 'IR_VJ_1_junction', 'IR_VJ

In [46]:
#sc.pp.neighbors(latent, n_neighbors=20, use_rep='X', method='gauss')
sc.pp.neighbors(adata, n_neighbors=5, method='gauss', knn=False)
sc.tl.diffmap(adata)
sc.tl.dpt(adata, n_branchings=1, n_dcs=10)

         Falling back to preprocessing with `sc.pp.pca` and default params.


In [94]:
k = 10
kf = KFold(n_splits=k) 
model = MultinomialNB()
def results_cross_validation(X, y):
    X = X.reshape(-1,1)
    acc_score = []
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        pred_values = model.predict(X_test)

        acc = accuracy_score(pred_values, y_test)
        acc_score.append(acc)

    avg_acc_score = sum(acc_score)/k
    return(acc_score, avg_acc_score)


In [114]:
le = LabelEncoder()
X = latent.obs['dpt_pseudotime'].to_numpy()
y = latent.obs['cell types'].to_numpy()
le.fit(y)
y = le.transform(y)
results1 = results_cross_validation(X,y)
print("Results for the latent space:")
print(" ")
print('accuracy of each fold - {}'.format(results1[0]))
print('Avg accuracy : {}'.format(results1[1]))

X = adata.obs['dpt_pseudotime'].to_numpy()
y = adata.obs['cell types'].to_numpy()
le.fit(y)
y = le.transform(y)
results2 = results_cross_validation(X,y)
print(" ")
print("Results for the adata space:")
print(" ")
print('accuracy of each fold - {}'.format(results2[0]))
print('Avg accuracy : {}'.format(results2[1]))


Results for the latent space:
 
accuracy of each fold - [0.22729347329986307, 0.25696029210406207, 0.20173436786855317, 0.17663167503423094, 0.1178082191780822]
Avg accuracy : 0.19608560549695828
 
Results for the adata space:
 
accuracy of each fold - [0.22729347329986307, 0.25696029210406207, 0.20173436786855317, 0.17663167503423094, 0.1178082191780822]
Avg accuracy : 0.19608560549695828


## Exactly the same method and results, but using cross_val_score:

In [116]:
X = latent.obs['dpt_pseudotime'].to_numpy()
X = X.reshape(-1,1)
y = latent.obs['cell types'].to_numpy()
le.fit(y)
y = le.transform(y)
k = 5
kf = KFold(n_splits=k, random_state=None)
model = MultinomialNB()
result = cross_val_score(model , X, y, cv = kf)
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.19608560549695828


In [117]:
X = adata.obs['dpt_pseudotime'].to_numpy()
X = X.reshape(-1,1)
y = adata.obs['cell types'].to_numpy()
le.fit(y)
y = le.transform(y)
k = 5
kf = KFold(n_splits=k, random_state=None)
model = MultinomialNB()
result = cross_val_score(model , X, y, cv = kf)
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.19608560549695828
